In [1]:
import pandas as pd
from numpy import array
import datetime
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define documents


Using TensorFlow backend.


In [2]:
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: {:.2%}'.format(accuracy))

[[40, 11], [12, 36], [17, 44], [13, 36], [31], [19], [17, 44], [14, 12], [17, 36], [6, 36, 11, 38]]
[[40 11  0  0]
 [12 36  0  0]
 [17 44  0  0]
 [13 36  0  0]
 [31  0  0  0]
 [19  0  0  0]
 [17 44  0  0]
 [14 12  0  0]
 [17 36  0  0]
 [ 6 36 11 38]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 80.00%


In [3]:
df = pd.read_csv('docs.csv').sample(frac=1).reset_index(drop=True)
df['date'] = df['rated'].apply(lambda x: datetime.datetime.strptime(x[:19],"%Y-%m-%d %H:%M:%S"))
df = df[df['date'] > datetime.datetime(2018,6,1,0,0,0)].reset_index(drop=True)
print(df.shape)
y = df['relevant']
df.head()

(1984, 13)


,Unnamed: 0,UT_id,authors,content,first_author,id,rated,relevant,tag,title,wosarticle__de,wosarticle__wc,date
0,3097,WOS:000364523100021,"Poehlein, A, Cebulla, M, Ilg, MM, Bengelsdorf,...",Clostridium aceticum was the first isolated au...,"Poehlein, A",581376,2018-09-21 07:54:18.039262+00:00,1,760,The Complete Genome Sequence of Clostridium ac...,NaN,['Microbiology'],2018-09-21 07:54:18
1,324,WOS:000276497900007,NaN,The increasing number of degraded soil areas c...,NaN,323125,2018-09-21 13:09:32.182834+00:00,1,760,Evaluating the Potential of Forest Species Und...,Cassiterite mine spoil; Amazon region; Arbuscu...,['Environmental Sciences; Meteorology & Atmosp...,2018-09-21 13:09:32
2,624,WOS:000207780600001,"Liao, IC",Asia is the largest continent in the world. As...,"Liao, IC",638275,2018-09-21 09:08:46.626102+00:00,1,760,Roles and contributions of fisheries science i...,fisheries science; Asia; capture fisheries; aq...,['Fisheries'],2018-09-21 09:08:46
3,3025,WOS:000297151300006,"Sharma, P, Salvato, C",Family firms vary with regards to success achi...,"Sharma, P",314899,2018-09-20 11:55:40.780222+00:00,0,760,Commentary: Exploiting and Exploring New Oppor...,NaN,['Business'],2018-09-20 11:55:40
4,527,WOS:000376697500015,"Lozano, R, Nummert, B, Ceulemans, K","An increasing number of companies have, during...","Lozano, R",187272,2018-09-21 02:51:10.396846+00:00,1,753,Elucidating the relationship between Sustainab...,Corporate sustainability; Sustainability Repor...,['GREEN & SUSTAINABLE SCIENCE & TECHNOLOGY; En...,2018-09-21 02:51:10


In [4]:
docs = array(df['content'])
labels = array(df['relevant'])

vocab_size = 10000
encoded_docs = [one_hot(d, vocab_size) for d in docs]

# pad documents to a max length of 4 words
max_length = 500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: {:.2%}'.format(accuracy))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 50001     
Total params: 1,050,001
Trainable params: 1,050,001
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 100.00%


In [5]:
docs = array(df['content'])
labels = array(df['relevant'])

vocab_size = 10000
encoded_docs = [one_hot(d, vocab_size) for d in docs]

# pad documents to a max length of 4 words
max_length = 500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')


import numpy as np
import sklearn.model_selection
X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(padded_docs, labels, random_state=1)

len(X_train)



1488

In [6]:
X_train.shape

(1488, 500)

In [ ]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, input_dim=200, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(X_train, y_train, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: {:.2%}'.format(accuracy))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 300)          3000000   
_________________________________________________________________
flatten_3 (Flatten)          (None, 150000)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 150001    
Total params: 3,150,001
Trainable params: 3,150,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model = Sequential() 
model.add(Embedding(vocab_size, 300, input_length=max_length)) 
model.add(LSTM(100)) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 

print(model.summary())
# fit the model
model.fit(X_train, y_train, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: {:.2%}'.format(accuracy))